In [ ]:
%matplotlib notebook

In [ ]:
import foldable_robotics
import foldable_robotics.dxf 
import numpy
import matplotlib.pyplot as plt
import shapely.geometry as sg
from foldable_robotics.layer import Layer
from foldable_robotics.laminate import Laminate
import foldable_robotics.manufacturing
import foldable_robotics.parts.castellated_hinge1

In [ ]:
filename = 'multilayer.dxf'

In [ ]:
def create_bodies(filename, layername, num_layers):
    body = foldable_robotics.dxf.read_lwpolylines(filename,layer=layername)

    bodies = [Layer(sg.Polygon(item)) for item in body]
    body = bodies.pop(0)
    for item in bodies:
        body ^= item
    body = body.to_laminate(num_layers)
    return body
    #body.plot()

In [ ]:
body_both = create_bodies(filename, 'body_both',11)
empty_layer = Layer()
empty_laminate_6 = Laminate(empty_layer,empty_layer,empty_layer,empty_layer,empty_layer,empty_layer)
body_upper = create_bodies(filename, 'body_upper',5)
body_lower = create_bodies(filename, 'body_lower',5)
body_upper = empty_laminate_6+body_upper
body_lower = body_lower+empty_laminate_6



In [ ]:
body_both.plot(new=True)

In [ ]:
body_lower.plot(new=True)

In [ ]:
body_upper.plot(new=True)

In [ ]:
body = body_both | body_lower|body_upper
body.plot()

In [ ]:
hinges_lower = foldable_robotics.dxf.read_lines(filename,layer='hinge_lower')
hinges_upper = foldable_robotics.dxf.read_lines(filename,layer='hinge_upper')


In [ ]:
def place_hinges(hinges_lower):
    castellated_hinge = foldable_robotics.parts.castellated_hinge1.generate()
    castellated_hinge = castellated_hinge.scale(1,.1)

    hinge1 = Layer().to_laminate(5)
    for p3,p4 in hinges_lower:
        hinge1|=castellated_hinge.map_line_stretch((0,0),(1,0),p3,p4)
    return hinge1

hinge1 = place_hinges(hinges_lower)
hinge1 = Laminate(*hinge1, *empty_laminate_6)
hinge2 = place_hinges(hinges_upper)
hinge2 = Laminate(*empty_laminate_6,*hinge2)

all_hinges = hinge1|hinge2
all_hinges.plot(new=True)

In [ ]:
device = body-all_hinges
device.plot(new=True)

In [ ]:
device_outer = foldable_robotics.manufacturing.unary_union(device)
sheet = (device_outer<<.5).bounding_box()
sheet=sheet.to_laminate(11)
device.plot(new=True)
sheet.plot()

In [ ]:
support_width = .1

In [ ]:
keepout =  foldable_robotics.manufacturing.keepout_laser(device)
second_pass_scrap = sheet-keepout
second_pass_scrap.plot(new=True)

Why is the center cut out of every hinge?

In [ ]:
first_pass_scrap = sheet - device-second_pass_scrap
first_pass_scrap.plot(new=True)
first_pass_scrap = foldable_robotics.manufacturing.cleanup(first_pass_scrap,.00001)
first_pass_scrap.plot(new=True)

In [ ]:
support = foldable_robotics.manufacturing.support(device,foldable_robotics.manufacturing.keepout_laser,support_width,support_width/2)
support.plot(new=True)

Calculate the web by using only the material which can be cut, minus a gap determined by the support width.  Is that the only material you can use?

In [ ]:
web = sheet-(keepout<<support_width)

In [ ]:
supported_design = web|device|support
supported_design.plot(new=True)

In [ ]:
first_pass_cut = sheet-supported_design
first_pass_cut.plot(new=True)

In [ ]:
is_adhesive = [False,True,False,True,False,True,False,True,False,True,False]

In [ ]:
#device.plot(new=True)
result = Layer()
Result = Laminate()
for layer in device:
    result |= layer
    Result += Laminate(result)
Result -=device  
Result.plot(new=True)
    

In [ ]:
#device.plot(new=True)
result = Layer()
Result = Laminate()
for layer in device[::-1]:
    result |= layer
    Result += Laminate(result)
Result = Result[::-1]
Result -=device  
#Result = foldable_robotics.manufacturing.modify_up(Result,is_adhesive)
Result.plot(new=True)

In [ ]:
nr_up = foldable_robotics.manufacturing.not_removable_up(device,is_adhesive)
#nr_up-=device
nr_up.plot(new=True)

In [ ]:

nr_both = foldable_robotics.manufacturing.not_removable_both(device)
nr_up = foldable_robotics.manufacturing.not_removable_up(device,is_adhesive)
nr_down = foldable_robotics.manufacturing.not_removable_down(device,is_adhesive)
second_pass_scrap = (nr_down & nr_up)-device
second_pass_scrap=sheet-second_pass_scrap
second_pass_scrap= foldable_robotics.manufacturing.cleanup(second_pass_scrap,.00001)
second_pass_scrap.plot(new=True)

In [ ]:
first_pass_scrap = sheet - device-second_pass_scrap
first_pass_scrap.plot(new=True)
first_pass_scrap = foldable_robotics.manufacturing.cleanup(first_pass_scrap,.00001)
first_pass_scrap.plot(new=True)

In [ ]:
supported_device = second_pass_scrap | device | support
supported_device.plot(new=True)
supported_device.plot_layers()

In [ ]:
first_pass = sheet-supported_device
first_pass.plot(new=True)